# Exercício: Previsão do preço de ações com LSTM (Tensorflow/Keras)
* **Descrição:** Carregar os preços de uma ação dos últimos x-anos (ex: 5). E tentar prever o próximo preço da ação
* **Dataset:** Puxar o preço histórico da ação de determinada empresa quer, via CSV ou utilizando **yfinance**
  * **Mais Informações:** https://pypi.org/project/yfinance/

## Passo a passo

1.   Descarregar dados dos últimos **X-anos** (ex: 5 anos) até hoje usando yfinance.
2. Manter apenas a coluna Close; aplicar MinMaxScaler para [0,1].
3. Criar sequências: 60 valores de fecho ➜ alvo = valor do dia seguinte.
4. Dividir 80 % treino / 20 % teste, preservando ordem temporal.
5. Modelo Keras: LSTM(50, return_sequences=True) + LSTM(50) + Dense(1).
6. compile(optimizer='adam', loss='mse'); treinar 25 épocas, batch 32.
7. Desfazer normalização; traçar real vs previsto; calcular RMSE.



In [ ]:
#!pip install yfinance pandas scikit-learn tensorflow matplotlib --quiet

import yfinance as yf, pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

In [ ]:
company = 'AAPL'
# 1. download do dataset (ou carregamento via CSV)
# Nesse caso usa a biblioteca yfinance que usa APIs públicas yahoo finance
df = yf.download(company, start='2020-04-01')
print('\n')
print(df.head())
df = df['Close']
sc = MinMaxScaler(feature_range=(0,1))
scaled = sc.fit_transform(df)

In [ ]:
# 2–3. janelas de 60 dias
seq_len = 60
X, y = [], []
for i in range(seq_len, len(scaled)):
    X.append(scaled[i-seq_len:i, 0])
    y.append(scaled[i, 0])
X, y = np.array(X), np.array(y)
X = X[..., None]             # (amostras, 60, 1)

In [ ]:
# 4. Divisão do dataset treino / teste
split = int(0.8*len(X))
Xtr, Xte, ytr, yte = X[:split], X[split:], y[:split], y[split:]

In [ ]:
# 5. modelo LSTM empilhado
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, return_sequences=True, input_shape=(seq_len,1)),
    tf.keras.layers.LSTM(50),
    tf.keras.layers.Dense(1)
])
model.compile('adam','mse')
model.fit(Xtr, ytr, epochs=25, batch_size=32, validation_split=0.1, verbose=0)

In [ ]:
# 6–7. previsão
pred_scaled = model.predict(Xte).flatten()
pred = sc.inverse_transform(pred_scaled.reshape(-1,1)).flatten()
real = sc.inverse_transform(yte.reshape(-1,1)).flatten()

rmse = np.sqrt(((pred - real)**2).mean())
print(f'RMSE = {rmse:.2f} USD')

In [ ]:
# Visualização
plt.figure(figsize=(10,4))
plt.plot(real, label='Real'); plt.plot(pred, label='Previsto')
plt.title(f'Preço de Fecho {company} — Real vs Previsto'); plt.legend(); plt.tight_layout()
plt.show()

In [ ]:
# Prever o preço para os próximos 5 dias
predicted_prices = []
current_sequence = scaled[-60:].reshape(1, 60, 1)

for _ in range(5):
  next_day_scaled = model.predict(current_sequence)
  next_day_price = sc.inverse_transform(next_day_scaled)[0,0]
  predicted_prices.append(next_day_price)

  # Atualizar a sequência para os próximos dias com os valores gerados
  current_sequence = np.concatenate((current_sequence[:,1:,:], next_day_scaled.reshape(1,1,1)), axis=1)

print("\nPreço real dos últimos 5 dias:")
for i in range(len(real[-5:])):
    print(f"{5 - i} dia(s) atrás: {real[-5:][i]:.2f}")


print("\nPrevisão de preço para os próximos 5 dias:")
for i, price in enumerate(predicted_prices):
    print(f"Daqui a {i+1} dia(s): {price:.2f}")
